# 8-9. 프로젝트 - Movielens 영화 추천 실습

---

## 0. 데이터 다운로드

In [50]:
# wget으로 데이터 다운로드
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2021-02-02 16:17:10--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  1.05MB/s    in 5.9s    

2021-02-02 16:17:16 (975 KB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [51]:
# 다운받은 데이터를 작업디렉토리로 옮김
!mv ml-1m.zip ~/aiffel/recommendata_iu/data

# 작업디렉토리로 이동
!cd ~/aiffel/recommendata_iu/data

#압축 해제
!unzip ml-1m.zip

unzip:  cannot find or open ml-1m.zip, ml-1m.zip.zip or ml-1m.zip.ZIP.


---

## 1. 데이터 준비와 전처리

In [52]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [53]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [54]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [55]:
ratings['count']
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [56]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


---

## 2. 데이터 분석

In [57]:
# 검색을 쉽게하기 위해 영화 제목 문자열을 소문자로 변경
movies.head()
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


### ratings에 있는 유니크한 영화 개수

In [58]:
ratings['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [59]:
ratings['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [60]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

---

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [61]:
# 내가 좋아하시는 영화 5가지 추가(단, 이름은 꼭 데이터셋에 있는 것과 동일)
# Mask, Bicentennial Man, Patch Adams, Sawshank Redemption, Toy Story
my_favorite = [367, 3156, 2431, 318, 1]

# 'ryu'라는 user_id가 위 영화 별점을 5점씩 주었다고 가정
my_movielist = pd.DataFrame({'user_id': ['ryu']*5, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['ryu']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10) 

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,ryu,367,5,NaN
1,ryu,3156,5,NaN
2,ryu,2431,5,NaN
3,ryu,318,5,NaN
4,ryu,1,5,NaN


In [63]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드(idx는 index의 약자)
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [65]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['ryu']) 
print(movie_to_idx[367])

6039
458


In [67]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 생성
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,458,5,NaN
1,6039,897,5,NaN
2,6039,640,5,NaN
3,6039,157,5,NaN


---

## 4. CSR matrix를 직접 만들어 봅시다.

In [71]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

---

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [73]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [74]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [75]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [76]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

---

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [78]:
ryu, mask = user_to_idx['ryu'], movie_to_idx[367]
ryu_vector, mask_vector = als_model.user_factors[ryu], als_model.item_factors[mask]

print('슝=3')

슝=3


In [79]:
ryu_vector

array([-0.7933157 ,  0.34778616,  0.1281192 ,  0.22634837, -0.3264717 ,
        0.07725845,  0.8946052 , -0.19709682, -0.53137857, -0.17390667,
       -0.39583585, -0.38675818, -0.21106568,  0.39315465,  0.14360206,
       -0.42560983,  0.27681798, -0.16465776, -0.9206708 , -0.32558274,
        0.5895571 ,  0.94288373, -0.5617389 ,  0.3097535 , -0.8488077 ,
        0.46713606, -0.57626426,  0.9395415 , -0.08579515,  0.6005603 ,
        0.49892852, -0.39071548,  0.3008018 ,  0.04483869,  0.44447297,
       -0.38392192, -0.14027642,  0.3079709 ,  0.12905394,  0.09810591,
        0.4016138 , -0.769246  ,  0.9446401 ,  0.19734932, -0.27672145,
        0.33833262, -0.13246709, -0.8931517 , -0.23694448,  0.32012814,
        1.3618548 , -0.11250485, -0.3396452 ,  0.7666896 ,  0.55273664,
       -0.5670847 , -0.26705268, -0.17053649,  0.2550878 , -0.39305755,
        1.287326  , -0.01911602,  0.28874138, -0.61875737,  0.48746797,
       -0.8027492 ,  1.0701641 ,  0.5844416 , -0.23047602,  0.53

In [80]:
mask_vector

array([-0.0032803 , -0.00148284, -0.00194756,  0.00411624,  0.00653218,
        0.00799281,  0.04065672,  0.00542972, -0.00524716,  0.02189304,
       -0.00121175,  0.01155992, -0.02255767,  0.0070994 ,  0.01219983,
        0.01722389, -0.0189208 , -0.00734517, -0.01580428,  0.02750188,
        0.03059313,  0.02914608, -0.00425296,  0.0166624 , -0.00439382,
        0.02202875, -0.01361296,  0.03869116,  0.01439934,  0.01390132,
        0.00093551,  0.00843411,  0.00129611,  0.02199772,  0.02975813,
        0.01320105,  0.02148682,  0.0164696 ,  0.00937807, -0.00069326,
       -0.00604525, -0.01417548, -0.01552557,  0.0157755 , -0.00432996,
        0.01268211,  0.01615132,  0.00854936, -0.02690962, -0.0055671 ,
        0.02928175,  0.01547258, -0.01025442,  0.01414802,  0.00921565,
        0.01012854, -0.00820857, -0.00666384,  0.00531288, -0.02870177,
        0.04800495,  0.01494214, -0.00155618, -0.02753159, -0.00698716,
       -0.00849672,  0.0225747 ,  0.00711561, -0.01380297,  0.00

In [81]:
# 내가 선호하는 5가지 영화 중 하나
# ryu과 mask를 내적하는 코드
np.dot(ryu_vector, mask_vector)

0.4278497

In [83]:
# 내가 선호하지 않았던 영화 중 하나
# ryu과 screamers를 내적하는 코드
screamers = movie_to_idx[76]
screamers_vector = als_model.item_factors[screamers]
np.dot(ryu_vector, screamers_vector)

0.023358349

---

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [86]:
movie_id = movie_to_idx[1]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0),
 (50, 0.82320136),
 (4, 0.5940365),
 (33, 0.5848552),
 (322, 0.5793053),
 (110, 0.5415979),
 (330, 0.47717768),
 (10, 0.4589531),
 (20, 0.44899994),
 (255, 0.40381953),
 (160, 0.36644876),
 (32, 0.35303634),
 (126, 0.34744772),
 (34, 0.34582773),
 (16, 0.32203394)]

In [87]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 2355,
 588,
 34,
 1265,
 364,
 595,
 2321,
 1923,
 356,
 1566,
 2396,
 1907,
 2687]

In [88]:
# 반복 확인을 위한 함수 생성
def get_similar_movie(movie_name: int):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [90]:
# 내가 좋아하는 다른 영화 "쇼생크 탈출"로 확인
get_similar_movie(318)

[318, 593, 527, 296, 608, 1704, 1213, 2028, 36, 50]

In [92]:
# 특정 장르(호러)로 확인
get_similar_movie(3340)

[3340, 2667, 2663, 2656, 2661, 2646, 2649, 2650, 2660, 3659]

---

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [93]:
user = user_to_idx['ryu']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.44848403),
 (4, 0.3775188),
 (248, 0.29604852),
 (545, 0.2941476),
 (121, 0.29333666),
 (48, 0.29315346),
 (322, 0.26026893),
 (51, 0.25316986),
 (222, 0.25220317),
 (110, 0.24980655),
 (23, 0.24734305),
 (81, 0.23890434),
 (812, 0.23048587),
 (269, 0.20505798),
 (33, 0.20302363),
 (626, 0.19435805),
 (475, 0.19347987),
 (139, 0.18981916),
 (596, 0.1876202),
 (244, 0.18077971)]

In [94]:
[idx_to_movie[i[0]] for i in movie_recommended]

[3114,
 2355,
 1704,
 317,
 593,
 2028,
 34,
 608,
 296,
 1265,
 527,
 3147,
 788,
 1213,
 588,
 2424,
 2302,
 2501,
 3489,
 1485]

In [95]:
glass_bottom_boat = movie_to_idx[3144]
explain = als_model.explain(user, csr_data, itemid=glass_bottom_boat)

In [97]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(2431, 0.00741369529626169),
 (3156, 0.0021002943125771707),
 (1, -0.004076112778352392),
 (318, -0.006441891704035939),
 (367, -0.00929444756817122)]

---

### 루브릭 평가

__1. CSR matrix가 정상적으로 만들어졌다.__  
_(사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.)_

__2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.__  
_(사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.)_

__3. 비슷한 영화찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.__  
_(MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.)_

---

## 회고

__1. 프로젝트 진행시 어려웠던 점__

__2. 프로젝트를 진행하면서 알아낸 점 또는 아직 모호한 점__

__3. 루브릭 평가 지표를 맞추기 위해 시도한 점__

__4. 루브릭 평가 지표를 달성하지 못한 이유__

__5. 프로젝트 진행 후 느낀 점 및 다짐__